In [10]:
import os 
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client, 
    set_default_openai_api,
    set_tracing_disabled,
    function_tool,
)
load_dotenv()

True

In [2]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

set_default_openai_client(client)
set_default_openai_api(
    ['chat_completions']
)
set_tracing_disabled(True)


In [16]:
import random

# TODO: add a strongly typed return value for the get_weather function 
# exmple: def get_weather(city: str) -> WeatherInfo:
# https://github.com/openai/openai-agents-python/blob/main/examples/basic/tools.py


class Weather(BaseModel):
    city: str
    temperature: str
    conditions: str

@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    temperature = f"{random.randint(-10, 50)}C"
    conditions = random.choice(["Sunny", "Windy", "Rainy", "Cloudy"])
    return Weather(city=city, temperature=temperature, conditions=conditions)    

In [17]:
class GitHubModelProvider(ModelProvider):
    def get_model(self, model_name) -> Model:
        return OpenAIChatCompletionsModel(model="gpt-4o", openai_client=client)

GITHUB_MODEL_PROVIDER = GitHubModelProvider()   

In [18]:
agent = Agent(
    name="Assistant", 
    instructions="Answer the user's questions.",
    tools=[get_weather],

)

In [19]:

"""This example uses a custom provider for some calls to Runner.run(), and direct calls to OpenAI for
others. Steps:
1. Create a custom OpenAI client.
2. Create a ModelProvider that uses the custom client.
3. Use the ModelProvider in calls to Runner.run(), only when we want to use the custom LLM provider.

Note that in this example, we disable tracing under the assumption that you don't have an API key
from platform.openai.com. If you do have one, you can either set the `OPENAI_API_KEY` env var
or call set_tracing_export_api_key() to set a tracing specific key.
"""

"This example uses a custom provider for some calls to Runner.run(), and direct calls to OpenAI for\nothers. Steps:\n1. Create a custom OpenAI client.\n2. Create a ModelProvider that uses the custom client.\n3. Use the ModelProvider in calls to Runner.run(), only when we want to use the custom LLM provider.\n\nNote that in this example, we disable tracing under the assumption that you don't have an API key\nfrom platform.openai.com. If you do have one, you can either set the `OPENAI_API_KEY` env var\nor call set_tracing_export_api_key() to set a tracing specific key.\n"

In [24]:
# https://github.com/openai/openai-agents-python/blob/main/examples/model_providers/custom_example_provider.py
result = await Runner.run(
    agent, 
    "What is the weather in New York?", 
    run_config=RunConfig(model_provider=GITHUB_MODEL_PROVIDER),
)

print(result.final_output)

[debug] getting weather for New York
The current weather in New York is 44°C with windy conditions.


In [25]:
result.last_agent

Agent(name='Assistant', instructions="Answer the user's questions.", handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=False, truncation=None, max_tokens=None), tools=[FunctionTool(name='get_weather', description='', params_json_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001F632685080>, strict_json_schema=True)], mcp_servers=[], input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)